# TriKirby Index

The TriKirby Index is a methodology inspired by the Kirby Index that quantifies the command and effectiveness of each UCSD Pitcher's pitches ahead of the 2026 season.

## Methodology: TriKirby Index (Percentile-Based Command Metric)

### Overview
The **TriKirby Index** is a pitch-level command metric designed to quantify a pitcher’s ability to consistently repeat release direction and release location. The metric extends the original Kirby Index framework by operating at the **pitch-type level**, normalizing performance **relative to NCAA Division I pitchers**, and producing an interpretable **percentile-based score**.

---

## 1. Pitch-Level Data Construction
All calculations begin from pitch-level TrackMan data. Each pitch includes measurements of:

- Vertical Release Angle (VRA)
- Horizontal Release Angle (HRA)
- Vertical Release Location ($vRel$)
- Horizontal Release Location ($hRel$)

Pitch-level data is retained until aggregation to preserve within-pitch variability.

---

## 2. Pitcher–Pitch Type Aggregation
For each pitcher $p$ and pitch type $t$, release consistency is summarized using the **standard deviation** of each release component:

$$
\sigma_{\text{VRA},p,t}, \quad
\sigma_{\text{HRA},p,t}, \quad
\sigma_{\text{vRel},p,t}, \quad
\sigma_{\text{hRel},p,t}
$$

Lower standard deviation indicates greater command consistency.

---

## 3. Percentile-Based Release Consistency Metrics

For each pitcher \( p \) and pitch type \( t \), release consistency is quantified using the
within-pitch-type **standard deviation** of four release components:

- Vertical Release Angle (VRA)
- Horizontal Release Angle (HRA)
- Vertical Release Location (vRel)
- Horizontal Release Location (hRel)

Lower variability indicates greater repeatability and improved pitch command.

To ensure comparability across NCAA Division I pitchers **within each pitch type**, each
standard deviation is converted into a percentile-based consistency score:

$$
\text{SD}_{p,t}^{\text{pct}} = 1 - \operatorname{rank}_{\text{pct}}\left(\text{SD}_{p,t}\right)
$$

This transformation ensures that:

- Scores lie in the interval \( [0,1] \)
- Higher values correspond to **better command**
- Each pitch type is normalized independently
- Metrics are robust to scale differences across pitch types

The resulting percentile-based release consistency metrics are:

$$
\text{sd\_vra\_pct}, \quad
\text{sd\_hra\_pct}, \quad
\text{sd\_vrel\_pct}, \quad
\text{sd\_hrel\_pct}
$$

---

## 4. Linear Component Weights

Each release component contributes differently to overall pitch command.
To reflect this, a set of **fixed linear weights** \( \beta_i \) is applied to each percentile-based
metric.

The weights are normalized such that:

$$
\sum_{i=1}^{4} \beta_i = 1
$$

These weights encode the relative importance of directional consistency
(release angles) versus spatial consistency (release location).

---
## 5. Feature Importance via Random Forest Regression

Each release component does not contribute equally to overall pitch command. To estimate the **relative importance** of each release dimension, the TriKirby Index uses a **Random Forest regression model** rather than fixed linear coefficients.

A Random Forest model is chosen because it:
- Captures **nonlinear relationships** between release mechanics and pitch location outcomes  
- Accounts for **interactions** between release variables  
- Avoids restrictive linear assumptions  

---

#### **Random Forest Model**

Let the feature vector for each pitch be:

$$
\mathbf{x} =
\left(
\text{VRA},\;
\text{HRA},\;
\text{vRel},\;
\text{hRel}
\right)
$$

A Random Forest regressor is trained to predict pitch location outcomes using these release features. After training, **feature importances** are extracted from the ensemble.

---

#### **Feature Importance Weights**

The Random Forest produces a set of feature importance weights:

$$
\boldsymbol{\beta}
=
\left(
\beta_{\text{VRA}},
\beta_{\text{HRA}},
\beta_{\text{vRel}},
\beta_{\text{hRel}}
\right)
$$

Each coefficient represents the **relative contribution** of a release component to pitch command, measured as the average reduction in prediction error across all trees.

By construction, the feature importances satisfy:

$$
\sum_{i=1}^{4} \beta_i = 1
$$

Thus, the weights are:
- **Already normalized**
- **Data-driven**
- **Nonlinear-aware**

No averaging or collapsing across plate dimensions is performed.

---

#### **Interpretation of Weights**

- Larger $\beta_i$ values indicate greater influence on pitch command  
- Weights reflect nonlinear and interaction-based effects  
- Directional consistency (release angles) and spatial consistency (release location) are allowed to contribute unequally  

These Random Forest–derived weights are used directly in the final TriKirby Index formulation.

---

## 6. TriKirby Index (Final Composite Score)

The **TriKirby Index** for pitcher $p$ and pitch type $t$ is computed as a weighted linear combination of percentile-based release consistency scores:

$$
\text{TriKirby}_{p,t}
=
\beta_{\text{VRA}} \cdot SD^{\text{pct}}_{\text{VRA},p,t}
+
\beta_{\text{HRA}} \cdot SD^{\text{pct}}_{\text{HRA},p,t}
+
\beta_{\text{vRel}} \cdot SD^{\text{pct}}_{\text{vRel},p,t}
+
\beta_{\text{hRel}} \cdot SD^{\text{pct}}_{\text{hRel},p,t}
$$

where:
- $SD^{\text{pct}} \in [0,1]$ denotes percentile-based consistency scores  
- Higher values correspond to better pitch command  
- Feature importance weights $\boldsymbol{\beta}$ are learned from the Random Forest model  


## 7. TriKirby Index (Final Score)

The **TriKirby Index** for pitcher \( p \) and pitch type \( t \) is computed as a weighted linear
combination of percentile-based release consistency metrics:

$$
\text{TriKirby}_{p,t}
= \beta_1 \cdot \text{sd\_vra\_pct}_{p,t}
+ \beta_2 \cdot \text{sd\_hra\_pct}_{p,t}
+ \beta_3 \cdot \text{sd\_vrel\_pct}_{p,t}
+ \beta_4 \cdot \text{sd\_hrel\_pct}_{p,t}
$$

---

## 8. Interpretation

- **Higher TriKirby values indicate better pitch command**
- Reflects tighter release direction and release point consistency
- Fully comparable **within pitch type** across NCAA Division I pitchers
- Designed for pitch-level evaluation rather than cross-pitch comparisons
- NCAA D1 Pitching Kirby Index Average Score across all pitch types is 0.5. Any values below 0.5 indicates a below average command on that pitch. Any values above 0.5 is above average command for that pitch across NCAA D1 baseball.


## 1. Data Preparation & Filtering

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

DATA_DIR = Path("Trackman CSVs")   

In [2]:
import pandas as pd
import numpy as np

# -------------------------------
# Load the two datasets
# -------------------------------
ncaa_df = pd.read_csv("all_games.csv")
ucsd_df = pd.read_csv("ucsd_fall_pitching_data.csv")  # <-- match your exact filename on the left

# -------------------------------
# Backward-compatible aliases
# (so the rest of the notebook works)
# -------------------------------
all_df = ncaa_df.copy()      # NCAA-wide (used for baselines / percentiles / betas)
ucsd_all = ucsd_df.copy()    # UCSD-only (used for your tables / rankings)

print("Loaded NCAA rows:", len(all_df), "| UCSD rows:", len(ucsd_all))
display(all_df.head(3))
display(ucsd_all.head(3))

/var/folders/zh/lt350y8j697d9gr_ryq4y0gm0000gn/T/ipykernel_11009/756047426.py:7: DtypeWarning: Columns (91,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  ncaa_df = pd.read_csv("all_games.csv")


Loaded NCAA rows: 1501313 | UCSD rows: 7268


,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,...,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence
0,1,2025-02-14,18:04:47.71,1,1,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Isom-McCall, Charlie",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
1,2,2025-02-14,18:05:00.16,1,2,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Isom-McCall, Charlie",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
2,3,2025-02-14,18:05:14.48,1,3,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Isom-McCall, Charlie",...,-78.35097,12.46061,High,High,High,NaN,NaN,Medium,Medium,Low


,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,...,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence
0,1,2025-11-01,11:35:12.81,1.0,1.0,"Gregson, Niccolas",1000039282,Right,CSD_TRI,"Crossland, Michael",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
1,2,2025-11-01,11:35:34.61,1.0,2.0,"Gregson, Niccolas",1000039282,Right,CSD_TRI,"Crossland, Michael",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
2,3,2025-11-01,11:36:01.08,1.0,3.0,"Gregson, Niccolas",1000039282,Right,CSD_TRI,"Crossland, Michael",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN


## 2. Pitch-Type–Specific Release Variability Metrics

In [3]:
df = all_df.copy()

def pick_col(candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

PITCHER_COL = pick_col(["Pitcher", "PitcherName", "PitcherNameFull"])
TEAM_COL    = pick_col(["PitcherTeam", "Team", "Pitcher Team", "PitcherTeamAbbrev"])
PITCHTYPE_COL = pick_col(["TaggedPitchType", "PitchType", "AutoPitchType"])

HRA_COL = pick_col(["HorzRelAngle"])
VRA_COL = pick_col(["VertRelAngle"])

# Horizontal + vertical release POINTS (usually X and Z)
RELX_COL = pick_col(["RelSide"])
RELZ_COL = pick_col(["RelHeight"])

print("Pitcher:", PITCHER_COL)
print("Team:", TEAM_COL)
print("PitchType:", PITCHTYPE_COL)
print("HRA:", HRA_COL)
print("VRA:", VRA_COL)
print("RelX:", RELX_COL)
print("RelZ:", RELZ_COL)

req = [PITCHER_COL, TEAM_COL, PITCHTYPE_COL, HRA_COL, VRA_COL, RELX_COL, RELZ_COL]
missing = [r for r in req if r is None]
if missing:
    raise ValueError("Missing required columns. Fix candidates list above. Missing: " + str(missing))

# Keep only what we need + clean
df = df.dropna(subset=[PITCHER_COL, TEAM_COL, PITCHTYPE_COL, HRA_COL, VRA_COL, RELX_COL, RELZ_COL]).copy()
df[PITCHTYPE_COL] = df[PITCHTYPE_COL].astype(str).str.strip()
df[PITCHER_COL]   = df[PITCHER_COL].astype(str).str.strip()
df[TEAM_COL]      = df[TEAM_COL].astype(str).str.strip()

df.head()

Pitcher: Pitcher
Team: PitcherTeam
PitchType: TaggedPitchType
HRA: HorzRelAngle
VRA: VertRelAngle
RelX: RelSide
RelZ: RelHeight


,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,...,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence
0,1,2025-02-14,18:04:47.71,1,1,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Isom-McCall, Charlie",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
1,2,2025-02-14,18:05:00.16,1,2,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Isom-McCall, Charlie",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
2,3,2025-02-14,18:05:14.48,1,3,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Isom-McCall, Charlie",...,-78.35097,12.46061,High,High,High,NaN,NaN,Medium,Medium,Low
3,4,2025-02-14,18:05:39.46,2,1,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Edwards, Kameron",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN
4,5,2025-02-14,18:05:53.55,2,2,"Stuprich, Brennan",1.000099e+09,Right,SOU_LIO,"Edwards, Kameron",...,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN


## UC San Diego Pitching

In [4]:
UCSD_CODE = "CSD_TRI"   # <-- based on what you saw in your output

ucsd_pitchers = sorted(df.loc[df[TEAM_COL] == UCSD_CODE, PITCHER_COL].unique())
print("UCSD pitchers found:", len(ucsd_pitchers))
ucsd_pitchers[:30]

UCSD pitchers found: 19


['Cazares, Julian',
 'Custer, Julian',
 'Dalquist, Matthew',
 'Davidson, Garrett',
 'Ernisse, Zach',
 'Gregson, Niccolas',
 'Hasegawa, Sam',
 'Huy, Nathan',
 'King, Devon',
 'Marchetti, Landon',
 'Murdock, Steele',
 'Nickerson, Trevor',
 'Patterson, Garrett',
 'Pelzman, Harry',
 'Remmers, Ethan',
 'Ries, Nathan',
 'Seid, Spencer',
 'Villar, Jake',
 'Weber, Chapman']

In [5]:
# === UCSD-only filter (do this BEFORE building spread) ===
TEAM_COL = "PitcherTeam"  # change if your df uses a different team column
UCSD_TEAM_NAMES = ["CSD_TRI", "UCSD", "SAN_DIEGO", "UC San Diego"]

print("Unique teams (sample):", df[TEAM_COL].dropna().astype(str).unique()[:20])

df = df[df[TEAM_COL].isin(UCSD_TEAM_NAMES)].copy()
print("Rows after UCSD filter:", len(df))

Unique teams (sample): ['SOU_LIO' 'LIN_UNI' 'OKL_COW' 'CLE_TIG' 'COL_CHA' 'VCU_RAM' 'ARK_RAZ'
 'WAS_COU' 'BAY_BEA' 'YSU_PEN' 'LON_DIR' 'NOR_CAT' 'GEO_BUL' 'QUI_BOB'
 'DIX_STE' 'NCB' 'CAL_BEA' 'NEV_WOL' 'ECU_PIR' 'GEO_PAT']
Rows after UCSD filter: 4459


## NCAA-Wide Spread Table

In [6]:
MIN_PITCHES_PER_TYPE = 0  # adjust if you want (prevents noisy tiny samples)

g = df.groupby([PITCHER_COL, PITCHTYPE_COL])

spread = g.agg(
    n_pitches=(PITCHTYPE_COL, "size"),
    sd_hra=(HRA_COL, "std"),
    sd_vra=(VRA_COL, "std"),
    sd_relx=(RELX_COL, "std"),
    sd_relz=(RELZ_COL, "std"),
).reset_index()

spread = spread[spread["n_pitches"] >= MIN_PITCHES_PER_TYPE].dropna()
spread.head()

,Pitcher,TaggedPitchType,n_pitches,sd_hra,sd_vra,sd_relx,sd_relz
0,"Cazares, Julian",ChangeUp,3,0.723429,0.262306,0.175095,0.146535
2,"Cazares, Julian",Fastball,38,0.721784,0.877410,0.146281,0.153933
3,"Cazares, Julian",FourSeamFastBall,3,2.076283,0.754681,0.086328,0.045497
4,"Cazares, Julian",Slider,29,1.095653,1.129368,0.143436,0.118094
5,"Cazares, Julian",TwoSeamFastBall,2,0.351325,0.062335,0.053669,0.023349


## NCAA Percentile-Based Command Scores

In [7]:
# Percentile-based command (higher = better)
for col in ["sd_hra", "sd_vra", "sd_relx", "sd_relz"]:
    spread[f"{col}_pct"] = 1 - spread.groupby(PITCHTYPE_COL)[col].rank(pct=True)

## Regression Model for Beta Weights

In [8]:
# Select Features + Target (Plate Location)

# Features (predictors)
FEATURES = [
    "VertRelAngle",   # VRA
    "HorzRelAngle",   # HRA
    "RelHeight",      # vRel
    "RelSide"         # hRel
]

# Targets (plate location)
TARGETS = [
    "PlateLocHeight",  # Z location
    "PlateLocSide"     # X location
]

model_df = df.dropna(subset=FEATURES + TARGETS)

X = model_df[FEATURES]
y = model_df[TARGETS]


In [9]:
# Test/Train Split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [10]:
# Linear Regression (Kirby-style)

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

y_pred = lin_reg.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))


MSE: 0.5178465982081192
R²: 0.41230259350869847


In [11]:
# Extract Beta Weights

beta_df = pd.DataFrame(
    lin_reg.coef_,
    columns=FEATURES,
    index=["PlateSide", "PlateHeight"]
)

beta_df

,VertRelAngle,HorzRelAngle,RelHeight,RelSide
PlateSide,0.365545,-0.046137,0.602566,-0.023966
PlateHeight,-0.046994,0.607446,0.542632,0.727966


In [12]:
beta_weights = beta_df.abs().mean(axis=0)
beta_weights = beta_weights / beta_weights.sum()  # normalize

beta_weights

VertRelAngle    0.139218
HorzRelAngle    0.220563
RelHeight       0.386467
RelSide         0.253752
dtype: float64

## Build Pitch Command Spread (Raw SDs)

In [13]:
import numpy as np

# --- 1) Identify the correct release point columns in YOUR CSV ---
# Trackman commonly uses RelHeight (vertical release) and RelSide (horizontal release)
CAND_VREL = ["RelHeight", "ReleaseHeight", "release_pos_z", "RelZ", "vRel"]
CAND_HREL = ["RelSide", "ReleaseSide", "release_pos_x", "RelX", "hRel"]

vrel_col = next((c for c in CAND_VREL if c in df.columns), None)
hrel_col = next((c for c in CAND_HREL if c in df.columns), None)

print("Using vRel column:", vrel_col)
print("Using hRel column:", hrel_col)

if vrel_col is None or hrel_col is None:
    raise ValueError(
        f"Could not find release point columns. "
        f"Columns in df include: {list(df.columns)[:40]} ..."
    )

# --- 2) Compute SDs per (Pitcher, PitchType) ---
PITCHER_COL = "Pitcher"
PITCHTYPE_COL = "TaggedPitchType"

spread = (
    df.groupby([PITCHER_COL, PITCHTYPE_COL])
      .agg(
          sd_vra=("VertRelAngle", lambda s: s.std(ddof=1)),
          sd_hra=("HorzRelAngle", lambda s: s.std(ddof=1)),
          sd_vrel=(vrel_col, lambda s: s.std(ddof=1)),
          sd_hrel=(hrel_col, lambda s: s.std(ddof=1)),
          n=("VertRelAngle", "size")
      )
      .reset_index()
)

# (Optional but recommended) drop tiny sample sizes to reduce noise
MIN_PITCHES_PER_TYPE = 0
spread = spread[spread["n"] >= MIN_PITCHES_PER_TYPE].copy()

# --- 3) Z-score each SD metric within pitch type (NCAA-wide benchmark) ---
def z_by_pitchtype(series):
    mu = series.mean()
    sd = series.std(ddof=0)
    return (series - mu) / sd if sd != 0 else np.nan

spread["z_sd_vra"]  = spread.groupby(PITCHTYPE_COL)["sd_vra"].transform(z_by_pitchtype)
spread["z_sd_hra"]  = spread.groupby(PITCHTYPE_COL)["sd_hra"].transform(z_by_pitchtype)
spread["z_sd_vrel"] = spread.groupby(PITCHTYPE_COL)["sd_vrel"].transform(z_by_pitchtype)
spread["z_sd_hrel"] = spread.groupby(PITCHTYPE_COL)["sd_hrel"].transform(z_by_pitchtype)

spread.head()

Using vRel column: RelHeight
Using hRel column: RelSide


,Pitcher,TaggedPitchType,sd_vra,sd_hra,sd_vrel,sd_hrel,n,z_sd_vra,z_sd_hra,z_sd_vrel,z_sd_hrel
0,"Cazares, Julian",ChangeUp,0.262306,0.723429,0.146535,0.175095,3,-2.078778,-0.441253,1.374577,-0.278438
1,"Cazares, Julian",Cutter,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
2,"Cazares, Julian",Fastball,0.877410,0.721784,0.153933,0.146281,38,-1.117055,-0.743044,0.675611,-0.536474
3,"Cazares, Julian",FourSeamFastBall,0.754681,2.076283,0.045497,0.086328,3,-1.030274,2.022747,-0.737004,-0.608437
4,"Cazares, Julian",Slider,1.129368,1.095653,0.118094,0.143436,29,-0.401259,-0.111984,0.227484,-0.391480


## NCAA-Normalized Release Consistency Percentiles

In [14]:
# ============================================================
# Convert release SDs to percentiles (lower SD = better)
# Done per pitch type (NCAA-normalized)
# Creates: sd_vra_pct, sd_hra_pct, sd_vrel_pct, sd_hrel_pct
# ============================================================

PCT_COLS = {
    "sd_vra":  "sd_vra_pct",
    "sd_hra":  "sd_hra_pct",
    "sd_vrel": "sd_vrel_pct",
    "sd_hrel": "sd_hrel_pct",
}

missing_raw = [c for c in PCT_COLS.keys() if c not in spread.columns]
if missing_raw:
    raise ValueError(f"Missing raw SD columns needed for percentiles: {missing_raw}")

for raw_col, pct_col in PCT_COLS.items():
    # rank within each pitch type; invert so lower SD = higher percentile (better)
    spread[pct_col] = 1 - spread.groupby(PITCHTYPE_COL)[raw_col].rank(pct=True)

# round for display (hundredths)
for c in PCT_COLS.values():
    spread[c] = spread[c].round(2)

display(spread[list(PCT_COLS.values())].describe())

,sd_vra_pct,sd_hra_pct,sd_vrel_pct,sd_hrel_pct
count,84.000000,84.000000,84.000000,84.000000
mean,0.434524,0.434524,0.434524,0.434524
std,0.296507,0.296507,0.296507,0.296507
min,0.000000,0.000000,0.000000,0.000000
25%,0.167500,0.167500,0.167500,0.167500
50%,0.440000,0.440000,0.440000,0.440000
75%,0.672500,0.672500,0.672500,0.672500
max,0.950000,0.950000,0.950000,0.950000


## Random Forest -> Feature Importance Betas

In [15]:
# ================================
# Random Forest: Feature Importances (Beta Weights)
# (nonlinear; already normalized to sum to 1)
# ================================

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

rf_regressor = RandomForestRegressor(
    n_estimators=300,
    random_state=42,
    n_jobs=-1,
    min_samples_leaf=5,
    max_features="sqrt"   # <-- FIX: don't use "auto"
)

rf_regressor.fit(X_train, y_train)

rf_importance = pd.Series(
    rf_regressor.feature_importances_,
    index=FEATURES
).sort_values(ascending=False)

# already sums to 1, but keep for safety
beta_weights = rf_importance / rf_importance.sum()

print("RF importances sum:", rf_importance.sum())
display(rf_importance)

print("Beta weights sum:", beta_weights.sum())
display(beta_weights)

RF importances sum: 1.0000000000000002


VertRelAngle    0.347710
HorzRelAngle    0.317594
RelSide         0.181851
RelHeight       0.152846
dtype: float64

Beta weights sum: 1.0


VertRelAngle    0.347710
HorzRelAngle    0.317594
RelSide         0.181851
RelHeight       0.152846
dtype: float64

## TriKirby Equation - Composite Pitch Command Metric

In [19]:
# TriKirby Equation (percentile-based, higher = better)

betas = {
    "sd_vra_pct": beta_weights["VertRelAngle"],
    "sd_hra_pct": beta_weights["HorzRelAngle"],
    "sd_vrel_pct": beta_weights["RelHeight"],
    "sd_hrel_pct": beta_weights["RelSide"],
}

spread["TriKirby"] = (
    betas["sd_vra_pct"] * spread["sd_vra_pct"] +
    betas["sd_hra_pct"] * spread["sd_hra_pct"] +
    betas["sd_vrel_pct"] * spread["sd_vrel_pct"] +
    betas["sd_hrel_pct"] * spread["sd_hrel_pct"]
).round(3)

display(
    spread[
        ["Pitcher", "TaggedPitchType", "TriKirby"]
    ].head()
)

,Pitcher,TaggedPitchType,TriKirby
0,"Cazares, Julian",ChangeUp,0.649
1,"Cazares, Julian",Cutter,NaN
2,"Cazares, Julian",Fastball,0.681
3,"Cazares, Julian",FourSeamFastBall,0.482
4,"Cazares, Julian",Slider,0.513


## UCSD TriKirby Index Score Across all Pitch Types (NCAA D1 AVG TriKirby Index Score = 0.5)

In [20]:
# ================================
# UCSD TriKirby by Pitch Type
# Percentile-based (0–1), higher = better
# ================================

ucsd_spread = spread.copy()

pitch_types = sorted(ucsd_spread[PITCHTYPE_COL].dropna().unique())
print("UCSD pitch types found:", pitch_types)

for pt in pitch_types:
    df_pt = (
        ucsd_spread[ucsd_spread[PITCHTYPE_COL] == pt]
        .sort_values("TriKirby", ascending=False)
        .copy()
    )

    display_cols = [
        PITCHER_COL,
        PITCHTYPE_COL,
        "n",
        "TriKirby"
    ]
    display_cols = [c for c in display_cols if c in df_pt.columns]

    print(f"\n=== {pt} (UCSD only) — TriKirby Command Percentile ===")
    display(df_pt[display_cols].reset_index(drop=True))

UCSD pitch types found: ['ChangeUp', 'Curveball', 'Cutter', 'Fastball', 'FourSeamFastBall', 'Knuckleball', 'Sinker', 'Slider', 'Splitter', 'Sweeper', 'TwoSeamFastBall']

=== ChangeUp (UCSD only) — TriKirby Command Percentile ===


,Pitcher,TaggedPitchType,n,TriKirby
0,"Marchetti, Landon",ChangeUp,10,0.658
1,"Dalquist, Matthew",ChangeUp,43,0.655
2,"Cazares, Julian",ChangeUp,3,0.649
3,"King, Devon",ChangeUp,4,0.639
4,"Gregson, Niccolas",ChangeUp,6,0.557
5,"Villar, Jake",ChangeUp,101,0.464
6,"Davidson, Garrett",ChangeUp,126,0.441
7,"Remmers, Ethan",ChangeUp,29,0.411
8,"Pelzman, Harry",ChangeUp,3,0.405
9,"Nickerson, Trevor",ChangeUp,7,0.403



=== Curveball (UCSD only) — TriKirby Command Percentile ===


,Pitcher,TaggedPitchType,n,TriKirby
0,"King, Devon",Curveball,9,0.654
1,"Dalquist, Matthew",Curveball,75,0.632
2,"Gregson, Niccolas",Curveball,58,0.622
3,"Remmers, Ethan",Curveball,3,0.592
4,"Marchetti, Landon",Curveball,14,0.448
5,"Hasegawa, Sam",Curveball,4,0.362
6,"Davidson, Garrett",Curveball,13,0.324
7,"Ries, Nathan",Curveball,55,0.222
8,"Villar, Jake",Curveball,16,0.145
9,"Murdock, Steele",Curveball,1,NaN



=== Cutter (UCSD only) — TriKirby Command Percentile ===


...

,Pitcher,TaggedPitchType,n,TriKirby
0,"Seid, Spencer",Splitter,18,0.0



=== Sweeper (UCSD only) — TriKirby Command Percentile ===


,Pitcher,TaggedPitchType,n,TriKirby
0,"Villar, Jake",Sweeper,24,0.333
1,"Seid, Spencer",Sweeper,5,0.167



=== TwoSeamFastBall (UCSD only) — TriKirby Command Percentile ===


,Pitcher,TaggedPitchType,n,TriKirby
0,"Cazares, Julian",TwoSeamFastBall,2,0.424
1,"King, Devon",TwoSeamFastBall,2,0.076


## TriKirby Index Score for Each Pitch Per UCSD Pitcher 

In [18]:
# ============================
# UCSD Pitcher Arsenal Tables
# (one DataFrame per pitcher)
# ============================

import pandas as pd

# --- 1) Identify the pitcher + pitchtype columns robustly ---
def pick_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

PITCHER_COL = pick_col(spread, ["Pitcher", "PitcherName", "PitcherNameFull"])
PITCHTYPE_COL = pick_col(spread, ["TaggedPitchType", "PitchType", "AutoPitchType"])
TEAM_COL = pick_col(spread, ["PitcherTeam", "Team", "Pitcher Team", "PitcherTeamAbbrev"])

# --- 2) UCSD filter (use TEAM_COL if available; otherwise fall back to name list if you have it) ---
if TEAM_COL is not None:
    UCSD_TEAM_NAMES = {"UCSD", "SAN_DIEGO", "UC San Diego", "UCSD_BASEBALL", "UCSD Baseball"}
    ucsd_spread = spread[spread[TEAM_COL].astype(str).str.strip().isin(UCSD_TEAM_NAMES)].copy()
else:
    # If you don't have a team column, use an explicit pitcher list you already have:
    # ucsd_spread = spread[spread[PITCHER_COL].isin(ucsd_pitchers)].copy()
    ucsd_spread = spread.copy()
    print("Warning: TEAM_COL not found. Using entire spread. If you have ucsd_pitchers list, filter here.")

# --- 3) Columns to show in each pitcher table ---
required = [PITCHER_COL, PITCHTYPE_COL, "n", "TriKirby"]
missing_req = [c for c in required if c not in ucsd_spread.columns or c is None]
if missing_req:
    raise ValueError(f"Missing required columns for pitcher tables: {missing_req}")

component_cols = [c for c in ["sd_vra_pct", "sd_hra_pct", "sd_vrel_pct", "sd_hrel_pct"] if c in ucsd_spread.columns]
show_cols = [PITCHTYPE_COL, "n", "TriKirby"] + component_cols

# --- 4) Build one DataFrame per pitcher (sorted by best TriKirby) ---
pitcher_tables = {}

for pitcher, df_p in ucsd_spread.groupby(PITCHER_COL):
    df_p = df_p.copy()

    # Sort: best TriKirby first, then more pitches
    df_p = df_p.sort_values(["TriKirby", "n"], ascending=[False, False])

    # Keep only columns we want (and make clean index)
    table = df_p[show_cols].reset_index(drop=True)

    # Optional: round for display
    if "TriKirby" in table.columns:
        table["TriKirby"] = table["TriKirby"].round(3)
    for c in component_cols:
        table[c] = table[c].round(2)

    pitcher_tables[pitcher] = table

# --- 5) Display each pitcher arsenal table ---
print(f"Built {len(pitcher_tables)} pitcher tables.")
for pitcher in sorted(pitcher_tables.keys()):
    print(f"\n=== {pitcher} — Arsenal (UCSD) ===")
    display(pitcher_tables[pitcher])

# --- 6) Optional: one combined long-form table (easy to export)
ucsd_by_pitcher_long = (
    ucsd_spread[[PITCHER_COL] + show_cols]
    .sort_values([PITCHER_COL, "TriKirby", "n"], ascending=[True, False, False])
    .reset_index(drop=True)
)

# Uncomment if you want to see it:
# display(ucsd_by_pitcher_long.head(50))

Built 19 pitcher tables.

=== Cazares, Julian — Arsenal (UCSD) ===


,TaggedPitchType,n,TriKirby,sd_vra_pct,sd_hra_pct,sd_vrel_pct,sd_hrel_pct
0,Fastball,38,0.681,0.84,0.79,0.21,0.58
1,ChangeUp,3,0.649,0.92,0.69,0.08,0.54
2,Slider,29,0.513,0.58,0.47,0.37,0.58
3,FourSeamFastBall,3,0.482,0.83,0.00,0.67,0.50
4,TwoSeamFastBall,2,0.424,0.50,0.50,0.00,0.50
5,Cutter,1,NaN,NaN,NaN,NaN,NaN



=== Custer, Julian — Arsenal (UCSD) ===


,TaggedPitchType,n,TriKirby,sd_vra_pct,sd_hra_pct,sd_vrel_pct,sd_hrel_pct
0,Fastball,5,0.820,0.63,0.89,0.95,0.95
1,Slider,3,0.281,0.16,0.16,0.95,0.16
2,Sinker,1,NaN,NaN,NaN,NaN,NaN



=== Dalquist, Matthew — Arsenal (UCSD) ===


...

,TaggedPitchType,n,TriKirby,sd_vra_pct,sd_hra_pct,sd_vrel_pct,sd_hrel_pct
0,Fastball,342,0.527,0.58,0.37,0.42,0.79
1,Slider,25,0.406,0.37,0.21,0.32,0.89
2,ChangeUp,35,0.282,0.15,0.46,0.00,0.46
3,Cutter,34,0.278,0.22,0.33,0.11,0.44
4,Sweeper,5,0.167,0.00,0.00,0.50,0.50
5,Splitter,18,0.000,0.00,0.00,0.00,0.00



=== Villar, Jake — Arsenal (UCSD) ===


,TaggedPitchType,n,TriKirby,sd_vra_pct,sd_hra_pct,sd_vrel_pct,sd_hrel_pct
0,ChangeUp,101,0.464,0.38,0.54,0.23,0.69
1,FourSeamFastBall,6,0.460,0.33,0.83,0.33,0.17
2,Fastball,159,0.450,0.26,0.84,0.05,0.47
3,Slider,123,0.430,0.21,0.63,0.53,0.42
4,Sinker,32,0.388,0.33,0.67,0.00,0.33
5,Sweeper,24,0.333,0.50,0.50,0.00,0.00
6,Curveball,16,0.145,0.00,0.11,0.33,0.33



=== Weber, Chapman — Arsenal (UCSD) ===


,TaggedPitchType,n,TriKirby,sd_vra_pct,sd_hra_pct,sd_vrel_pct,sd_hrel_pct
0,Slider,25,0.622,0.32,0.89,0.74,0.63
1,Fastball,199,0.477,0.42,0.42,0.79,0.42
2,ChangeUp,1,NaN,NaN,NaN,NaN,NaN
